<a href="https://colab.research.google.com/github/RuolinZheng08/cmsc25460-optimization/blob/master/Homework2_lp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2 

[CVXPY](https://www.cvxpy.org/tutorial/intro/index.html) is a Python-embedded modeling language for convex optimization problems. It allows you to express your problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

# Exercise

### Shortest s-t path

Use CVXPY to construct and solve the linear programming formulation of $s$-$t$ shortest path from Problem 5 over the graph below, where edges are labeled by their length.

![Flow Graph](https://drive.google.com/uc?export=view&id=1NOZuGFUkXOO780FQsMzNsvPDe4udnzHJ)

Submit your code and your solution (including primal and dual optimal solutions) as a pdf through gradescope with the rest of the homework.

__Note__: There are two strategies to accomplish this. The simplest one is to explicitly write all the constraints for the linear program algebraically as done in the examples you have seen before. If you are more comfortable with Python, you could try to use the object oriented approach described [here](https://www.cvxpy.org/examples/applications/OOCO.html).

In [0]:
import cvxpy as cp

edges = [   # (u, v, w)
    ("S", "A", 6),
    ("S", "B", 1),
    ("S", "C", 10),
    ("A", "B", 2),
    ("A", "D", 4),
    ("A", "E", 1),
    ("B", "E", 20),
    ("C", "B", 2),
    ("C", "F", 5),
    ("D", "E", 2),
    ("D", "G", 5),
    ("E", "F", 6),
    ("E", "G", 10),
    ("F", "T", 4),
    ("G", "T", 12)
]

